In [1]:
!pip install faiss-cpu transformers datasets sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing i

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import faiss
import torch
from transformers import (
    AutoTokenizer, AutoModelForQuestionAnswering,
    pipeline, AutoModelForCausalLM
)
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import numpy as np


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

2025-06-15 04:14:07.601797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749960847.811205      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749960847.871273      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


/kaggle/input/pubmed-data/PubMed_data.csv


In [3]:
#read in the data 
pubmed = pd.read_csv('/kaggle/input/pubmed-data/PubMed_data.csv')
pubmed.head()

,PMID,Title,Abstract,Authors,Journal,Keywords,URL,Affiliations
0,38133628,Age-related micro-environmental changes as dri...,Both aging and reduced diversity at the hemato...,"Bacharach Tal, Kaushansky Nathali, Shlush Liran I",Current opinion in hematology,"Humans, Female, Clonal Hematopoiesis, Hematopo...",https://www.ncbi.nlm.nih.gov/pubmed/38133628,"Department of Molecular Cell Biology, Weizmann..."
1,38161058,A Novel Nonhormonal Treatment for Vasomotor Sy...,"Vasomotor symptoms of menopause, more commonly...",Javernick Julie A,Nursing for women's health,"Female, Humans, Menopause, Hot Flashes, Hetero...",https://www.ncbi.nlm.nih.gov/pubmed/38161058,NaN
2,38160985,Endometriosis and mental health disorders: ide...,Endometriosis is a disease marked by more than...,"Zippl Anna Lena, Reiser Elisabeth, Seeber Beata",Fertility and sterility,"Humans, Female, Endometriosis, Quality of Life...",https://www.ncbi.nlm.nih.gov/pubmed/38160985,Department of Gynecological Endocrinology and ...
3,38160795,Validation of the Canadian Institute for Healt...,We investigated the validity of the 10th Revis...,"Payne Magdalene, Bacal Vanessa, Bougie Olga, N...",Journal of obstetrics and gynaecology Canada :...,"Humans, Female, Canada, International Classifi...",https://www.ncbi.nlm.nih.gov/pubmed/38160795,"Department of Obstetrics and Gynecology, Queen..."
4,38160748,Combined Robotic Transanal Transection Single-...,To describe a combined robotic and transanal t...,"D'Ancona Gianmarco, Merlot Benjamin, Denost Qu...",Journal of minimally invasive gynecology,"Female, Humans, Adult, Endometriosis, Robotic ...",https://www.ncbi.nlm.nih.gov/pubmed/38160748,Franco-European Multidisciplinary Endometriosi...


In [4]:
##Check to see if there are any null values (only care if there are nulls in the title/abstract/authors)
pubmed.isnull().sum()

PMID             0
Title            0
Abstract         0
Authors          0
Journal          0
Keywords        67
URL              0
Affiliations    15
dtype: int64

In [5]:
##Dataset pre-processing: have to create "context" out of my current dataset: 
def create_context(row):
    authors = row['Authors'] if 'Authors' in row and pd.notna(row['Authors']) else ""
    title = row['Title'] if 'Title' in row and pd.notna(row['Title']) else ""
    abstract = row['Abstract'] if 'Abstract' in row and pd.notna(row['Abstract']) else ""
    combined = f"Title: {title} \n Authors: {authors} \n Abstract: {abstract}"
    return combined

pubmed['context'] = pubmed.apply(create_context, axis=1)
documents = pubmed['context'].tolist()
print(f"Loaded {len(documents)} documents.")

Loaded 276 documents.


In [6]:
#check a full context 
pubmed['context'].iloc[0]

'Title: Age-related micro-environmental changes as drivers of clonal hematopoiesis. \n Authors: Bacharach Tal, Kaushansky Nathali, Shlush Liran I \n Abstract: Both aging and reduced diversity at the hematopoietic stem cells (HSCs) level are ubiquitous. What remains unclear is why some individuals develop clonal hematopoiesis (CH), and why does CH due to specific mutations occur in specific individuals. Much like aging, reduced diversity of HSCs is a complex phenotype shaped by numerous factors (germline & environment). The purpose of the current review is to discuss the role of two other age-related ubiquitous processes that might contribute to the dynamics and characteristics of losing HSC diversity and the evolution of CH. These processes have not been reviewed in depth so far and include the accumulation of fatty bone marrow (FBM), and the decline in sex hormones. Interestingly, sex hormone decline can directly shape HSC function, but also reshape the delicate balance of BM supporti

In [7]:
##create embeddings:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
doc_embeddings = embedder.encode(documents, convert_to_tensor=False)
doc_embeddings = np.array(doc_embeddings).astype("float32")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
## Build FAISS index:
index = faiss.IndexFlatL2(doc_embeddings.shape[1])
index.add(doc_embeddings)


In [9]:
## Load BioBERT QA model
# qa_model_name = "dmis-lab/biobert-base-cased-v1.1"
# qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
# qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

# qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)
qa_model_name = "ktrapeznikov/biobert_v1.1_pubmed_squad_v2"
qa_tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
qa_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

qa_pipeline = pipeline("question-answering", model=qa_model, tokenizer=qa_tokenizer)


tokenizer_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at ktrapeznikov/biobert_v1.1_pubmed_squad_v2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

In [10]:
##retrieve top-k relevant docs:
def retrieve_relevant_docs(question, k=3):
    question_embedding = embedder.encode([question])[0].astype("float32")
    _, doc_indices = index.search(np.array([question_embedding]), k)
    return [documents[i] for i in doc_indices[0]]


In [11]:
# Answer question with biobert
def answer_with_biobert(question, docs, top_n=1):
    results = []
    for doc in docs:
        result = qa_pipeline({"question": question, "context": doc})
        results.append((result['answer'], result['score']))
    results.sort(key=lambda x: x[1], reverse=True)
    return results[:top_n]


In [12]:
##Need credentials to use Mistral
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("mistral-token")


# Paste your Hugging Face token here (replace with your actual token)
login(secret_value_0)


In [13]:
##Load mistral for rephrasing
mistral_name = "mistralai/Mistral-7B-Instruct-v0.2"

mistral_tokenizer = AutoTokenizer.from_pretrained(mistral_name)
mistral_model = AutoModelForCausalLM.from_pretrained(
    mistral_name,
    device_map="auto",
    torch_dtype=torch.float16
)

rephrase_pipeline = pipeline(
    "text-generation",
    model=mistral_model,
    tokenizer=mistral_tokenizer
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
## Rephrase BioBert answer:
def rephrase_answer(factual_answer, question):
    prompt = (
        f"You are a helpful AI health assistant. Based on the following question and answer, "
        f"rephrase the answer in a clear, friendly way.\n\n"
        f"Question: {question}\n"
        f"Answer: {factual_answer}\n\n"
        f"Rephrased:"
    )
    output = rephrase_pipeline(prompt, max_new_tokens=80, temperature=0.7)
    return output[0]["generated_text"].split("Rephrased:")[-1].strip()


In [15]:
## Full ask-and-rephrase pipeline:
def ask_and_rephrase(question):
    docs = retrieve_relevant_docs(question, k=3)
    raw_answers = answer_with_biobert(question, docs, top_n=1)

    if not raw_answers:
        return "Sorry, I couldn't find an answer."

    factual_answer, confidence = raw_answers[0]
    friendly_answer = rephrase_answer(factual_answer, question)

    return {
        "original": factual_answer,
        "confidence": confidence,
        "rephrased": friendly_answer
    }


In [16]:
#Test:
question = "What treatments are recommended for PCOS?"
response = ask_and_rephrase(question)

print("🔬 BioBERT Answer:", response["original"])
print("🧠 Confidence Score:", round(response["confidence"], 3))
print("🤖 Rephrased Answer:", response["rephrased"])


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


🔬 BioBERT Answer: Lifestyle strategies
🧠 Confidence Score: 0.851
🤖 Rephrased Answer: For PCOS, there are several effective treatments you might consider. One of the first steps is making some lifestyle changes. These can include eating a healthy diet, getting regular physical activity, and managing stress. These simple adjustments can help improve your symptoms and overall health. If lifestyle changes aren't enough, your doctor may recommend additional treatments like medication or hormone therapy. But remember, every


In [17]:
## Give just the rephrased output
def final_output(question): 
    whole_response = ask_and_rephrase(question)
    final_rephrased_answer = whole_response['rephrased']
    return final_rephrased_answer

In [18]:
#Test 2
question = 'How is PCOS diagnosed?'
final_output(question)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'A physician is the one who can diagnose Polycystic Ovary Syndrome (PCOS) based on specific symptoms and diagnostic tests, such as blood tests to check hormone levels and ultrasound to examine the ovaries.'

In [19]:
#Test 3 
question = 'What are the signs of endometriosis?'
print(final_output(question))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Endometriosis can present with various symptoms. These may include:

1. Severe menstrual cramps and pelvic pain
2. Heavy menstrual bleeding
3. Pain during or after sex
4. Infertility
5. Painful bowel movements or urination
6. Fatigue, bloating, or diarrhea
7


In [20]:
##TODO: The output isn't perfect, as seen by the extra '7' above. Do some fine-tuning of the rephrase_pipeline to get around this